# Environment setup and data preprocessing

In [20]:
# Install and Import Required Libraries
#!pip install -q pandas numpy matplotlib seaborn scikit-learn xgboost scipy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import scipy
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the HVAC Energy Dataset
# The dataset has been uploaded to Colab with the filename 'HVAC Energy Data.csv'

df = pd.read_csv('dataset/HVAC Energy Data.csv')  # Loading the dataset

print(f"\nDataset shape: {df.shape}")
print(f"Number of samples: {df.shape[0]}")
print(f"Number of features: {df.shape[1]}")

# Display first few rows
print("\nFirst 5 rows of the dataset:")
print(df.head())


Dataset shape: (13615, 10)
Number of samples: 13615
Number of features: 10

First 5 rows of the dataset:
  Local Time (Timezone : GMT+8h)  Chilled Water Rate (L/sec)  \
0                 8/18/2019 0:00                        85.6   
1                 8/18/2019 0:30                        84.3   
2                 8/18/2019 1:00                        84.6   
3                 8/18/2019 1:30                        84.8   
4                 8/18/2019 2:00                        84.7   

   Cooling Water Temperature (C)  Building Load (RT)  \
0                           31.4               479.6   
1                           31.1               460.3   
2                           31.3               462.4   
3                           31.4               458.5   
4                           31.4               455.4   

   Chiller Energy Consumption (kWh)  Outside Temperature (F)  Dew Point (F)  \
0                             116.2                       82             75   
1             

In [3]:
# Inspect Dataset Structure

print("=" * 60)
print("DATASET INFORMATION")
print("=" * 60)
print(df.info())

print("\n" + "=" * 60)
print("STATISTICAL SUMMARY")
print("=" * 60)
print(df.describe())

print("\n" + "=" * 60)
print("COLUMN NAMES")
print("=" * 60)
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

DATASET INFORMATION
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13615 entries, 0 to 13614
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Local Time (Timezone : GMT+8h)    13615 non-null  object 
 1   Chilled Water Rate (L/sec)        13615 non-null  float64
 2   Cooling Water Temperature (C)     13615 non-null  float64
 3   Building Load (RT)                13615 non-null  float64
 4   Chiller Energy Consumption (kWh)  13615 non-null  float64
 5   Outside Temperature (F)           13615 non-null  int64  
 6   Dew Point (F)                     13615 non-null  int64  
 7   Humidity (%)                      13615 non-null  int64  
 8   Wind Speed (mph)                  13615 non-null  int64  
 9   Pressure (in)                     13615 non-null  float64
dtypes: float64(5), int64(4), object(1)
memory usage: 1.0+ MB
None

STATISTICAL SUMMARY
       Chilled Water Rate (

In [4]:
# Data Cleaning - Handle Missing Values and Verify Outlier Removal

print("=" * 60)
print("CHECKING FOR MISSING VALUES")
print("=" * 60)
missing_values = df.isnull().sum()
print(missing_values)
print(f"\nTotal missing values: {df.isnull().sum().sum()}")

# Handle any missing values if present (forward fill or interpolation)
if df.isnull().sum().sum() > 0:
    print("\nMissing values detected. Applying forward fill...")
    df = df.fillna(method='ffill')
    print("Missing values handled!")
else:
    print("\nNo missing values detected. Dataset is clean!")


CHECKING FOR MISSING VALUES
Local Time (Timezone : GMT+8h)      0
Chilled Water Rate (L/sec)          0
Cooling Water Temperature (C)       0
Building Load (RT)                  0
Chiller Energy Consumption (kWh)    0
Outside Temperature (F)             0
Dew Point (F)                       0
Humidity (%)                        0
Wind Speed (mph)                    0
Pressure (in)                       0
dtype: int64

Total missing values: 0

No missing values detected. Dataset is clean!


In [5]:
# Verify outlier removal by checking statistical ranges
print("\n" + "=" * 60)
print("OUTLIER VERIFICATION (Using IQR Method)")
print("=" * 60)
for col in df.select_dtypes(include=[np.number]).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col].count()
    print(f"{col}: {outliers} potential outliers detected")

print(f"\nData cleaning completed! Final dataset shape: {df.shape}")


OUTLIER VERIFICATION (Using IQR Method)
Chilled Water Rate (L/sec): 15 potential outliers detected
Cooling Water Temperature (C): 60 potential outliers detected
Building Load (RT): 23 potential outliers detected
Chiller Energy Consumption (kWh): 1081 potential outliers detected
Outside Temperature (F): 14 potential outliers detected
Dew Point (F): 15 potential outliers detected
Humidity (%): 26 potential outliers detected
Wind Speed (mph): 2 potential outliers detected
Pressure (in): 363 potential outliers detected

Data cleaning completed! Final dataset shape: (13615, 10)


In [6]:
# Convert Timestamp to Datetime and Set as Index

print("=" * 60)
print("TIMESTAMP CONVERSION")
print("=" * 60)

# Convert 'Local Time' column to datetime format
timestamp_column = 'Local Time (Timezone : GMT+8h)'
df[timestamp_column] = pd.to_datetime(df[timestamp_column])

print(f"\nTimestamp column '{timestamp_column}' converted to datetime format")
print(f"Timestamp data type: {df[timestamp_column].dtype}")
print(f"Date range: {df[timestamp_column].min()} to {df[timestamp_column].max()}")

# Set timestamp as index for time series analysis
df = df.set_index(timestamp_column)

print(f"\nTimestamp set as index successfully!")
print(f"\nDataset after setting index:")
print(df.head())
print(f"\nIndex type: {type(df.index)}")
print(f"Dataset shape: {df.shape}")

TIMESTAMP CONVERSION

Timestamp column 'Local Time (Timezone : GMT+8h)' converted to datetime format
Timestamp data type: datetime64[ns]
Date range: 2019-08-18 00:00:00 to 2020-06-01 13:00:00

Timestamp set as index successfully!

Dataset after setting index:
                                Chilled Water Rate (L/sec)  \
Local Time (Timezone : GMT+8h)                               
2019-08-18 00:00:00                                   85.6   
2019-08-18 00:30:00                                   84.3   
2019-08-18 01:00:00                                   84.6   
2019-08-18 01:30:00                                   84.8   
2019-08-18 02:00:00                                   84.7   

                                Cooling Water Temperature (C)  \
Local Time (Timezone : GMT+8h)                                  
2019-08-18 00:00:00                                      31.4   
2019-08-18 00:30:00                                      31.1   
2019-08-18 01:00:00                          

In [7]:
# Min-Max Normalization to Scale Features Between 0 and 1

print("=" * 60)
print("MIN-MAX NORMALIZATION")
print("=" * 60)

# Store the original column names
feature_columns = df.columns.tolist()

print(f"\nFeatures to normalize: {len(feature_columns)}")
for i, col in enumerate(feature_columns, 1):
    print(f"{i}. {col}")

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply normalization to all features
df_normalized = pd.DataFrame(
    scaler.fit_transform(df),
    columns=feature_columns,
    index=df.index
)

print(f"\nMin-Max Normalization applied successfully!")
print(f"\nAll features are now scaled between 0 and 1.")

# Display normalized data statistics
print("\n" + "=" * 60)
print("NORMALIZED DATA STATISTICS")
print("=" * 60)
print(df_normalized.describe())

print("\n" + "=" * 60)
print("SAMPLE OF NORMALIZED DATA")
print("=" * 60)
print(df_normalized.head(10))

print(f"\nNormalized dataset shape: {df_normalized.shape}")

MIN-MAX NORMALIZATION

Features to normalize: 9
1. Chilled Water Rate (L/sec)
2. Cooling Water Temperature (C)
3. Building Load (RT)
4. Chiller Energy Consumption (kWh)
5. Outside Temperature (F)
6. Dew Point (F)
7. Humidity (%)
8. Wind Speed (mph)
9. Pressure (in)

Min-Max Normalization applied successfully!

All features are now scaled between 0 and 1.

NORMALIZED DATA STATISTICS
       Chilled Water Rate (L/sec)  Cooling Water Temperature (C)  \
count                13615.000000                   13615.000000   
mean                     0.352254                       0.559834   
std                      0.181750                       0.120509   
min                      0.000000                       0.000000   
25%                      0.209841                       0.480769   
50%                      0.315485                       0.548077   
75%                      0.487699                       0.644231   
max                      1.000000                       1.000000   

  

In [8]:
# Step 7: Split Data into Training (80%) and Testing (20%) Sets Chronologically

print("=" * 60)
print("TRAIN-TEST SPLIT (CHRONOLOGICAL)")
print("=" * 60)

# Calculate split index for 80-20 split
total_samples = len(df_normalized)
train_size = int(0.8 * total_samples)

print(f"\nTotal samples: {total_samples}")
print(f"Training samples (80%): {train_size}")
print(f"Testing samples (20%): {total_samples - train_size}")

# Split data chronologically (maintaining temporal order)
train_data = df_normalized.iloc[:train_size]
test_data = df_normalized.iloc[train_size:]

print("\nData split completed successfully!")

print("\n" + "=" * 60)
print("TRAINING SET DETAILS")
print("=" * 60)
print(f"Shape: {train_data.shape}")
print(f"Date range: {train_data.index.min()} to {train_data.index.max()}")
print(f"\nFirst few rows:")
print(train_data.head())

print("\n" + "=" * 60)
print("TESTING SET DETAILS")
print("=" * 60)
print(f"Shape: {test_data.shape}")
print(f"Date range: {test_data.index.min()} to {test_data.index.max()}")
print(f"\nFirst few rows:")
print(test_data.head())

TRAIN-TEST SPLIT (CHRONOLOGICAL)

Total samples: 13615
Training samples (80%): 10892
Testing samples (20%): 2723

Data split completed successfully!

TRAINING SET DETAILS
Shape: (10892, 9)
Date range: 2019-08-18 00:00:00 to 2020-04-05 18:30:00

First few rows:
                                Chilled Water Rate (L/sec)  \
Local Time (Timezone : GMT+8h)                               
2019-08-18 00:00:00                               0.191027   
2019-08-18 00:30:00                               0.172214   
2019-08-18 01:00:00                               0.176556   
2019-08-18 01:30:00                               0.179450   
2019-08-18 02:00:00                               0.178003   

                                Cooling Water Temperature (C)  \
Local Time (Timezone : GMT+8h)                                  
2019-08-18 00:00:00                                  0.538462   
2019-08-18 00:30:00                                  0.509615   
2019-08-18 01:00:00                         

In [9]:
# Save the cleaned and splitted dataset
df_normalized.to_csv('dataset/HVAC Energy Data Cleaned and Split.csv')